In [4]:
!git clone https://github.com/google-deepmind/materials_discovery.git

!pip install pymatgen


from typing import List, Tuple

import itertools
import json
import os
import pandas as pd

import pymatgen as mg
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.analysis import phase_diagram
PUBLIC_LINK = "https://storage.googleapis.com/"
BUCKET_NAME = "gdm_materials_discovery"

FOLDER_NAME = "gnome_data"
FILES = (
    "stable_materials_summary.csv",
)

EXTERNAL_FOLDER_NAME = "external_data"
EXTERNAL_FILES = (
    "external_materials_summary.csv",
)

def download_from_link(link: str, output_dir: str):
  """Download a file from a public link using wget."""
  os.system(f"wget {link} -P {output_dir}")

parent_directory = os.path.join(PUBLIC_LINK, BUCKET_NAME)
for filename in FILES:
  public_link = os.path.join(parent_directory, FOLDER_NAME, filename)
  download_from_link(public_link, '.')

for filename in EXTERNAL_FILES:
  public_link = os.path.join(parent_directory, EXTERNAL_FOLDER_NAME, filename)
  download_from_link(public_link, '.')

gnome_crystals = pd.read_csv('stable_materials_summary.csv', index_col=0)
gnome_crystals
reference_crystals = pd.read_csv('external_materials_summary.csv')
reference_crystals

def annotate_chemical_system(crystals: pd.DataFrame) -> pd.DataFrame:
  chemical_systems = []
  for i, e in enumerate(crystals['Elements']):
    try:
      # replace single quotes with double quotes to avoid having to use python eval
      chemsys = json.loads(e.replace("'", '"'))
      chemical_systems.append(tuple(sorted(chemsys)))
    except:
      print(e)
  crystals['Chemical System'] = chemical_systems
  return crystals


# Preprocess crystal structure
gnome_crystals = annotate_chemical_system(gnome_crystals)
reference_crystals = annotate_chemical_system(reference_crystals)


all_crystals = pd.concat([gnome_crystals, reference_crystals], ignore_index=True)
required_columns = ['Composition', 'NSites', 'Corrected Energy', 'Formation Energy Per Atom', 'Chemical System']
minimal_entries = all_crystals[required_columns]
grouped_entries = minimal_entries.groupby('Chemical System')
# Choose a sample crystal
binaries = gnome_crystals[gnome_crystals['Chemical System'].map(len) == 2]
sample = binaries.sample()


# The sample we have chosen and would like to visualize
sample
chemsys = sample['Chemical System'].item()
from pymatgen.analysis import interface_reactions


element = 'O'
temperature = 300
pressure = 21200
chempot_oxygen = interface_reactions.InterfacialReactivity.get_chempot_correction(
    element, temperature, pressure)
u_o = -4.95 + chempot_oxygen


oxygen_chemsys = chemsys + ('O',)
chempots = {mg.core.Element('O'): u_o}


def collect_phase_diagram_entries(
    chemsys: Tuple[str, ...],
    grouped_entries: pd.core.groupby.generic.DataFrameGroupBy,
    minimal_entries: pd.DataFrame
) -> List[ComputedEntry]:
  phase_diagram_entries = []
  for length in range(len(chemsys) + 1):
    for subsystem in itertools.combinations(chemsys, length):
      subsystem_key = tuple(sorted(subsystem))
      subsystem_entries = grouped_entries.groups.get(subsystem_key, [])
      if len(subsystem_entries):
        phase_diagram_entries.append(minimal_entries.iloc[subsystem_entries])
  phase_diagram_entries = pd.concat(phase_diagram_entries)

  mg_entries = []

  for _, row in phase_diagram_entries.iterrows():
    composition = row['Composition']
    formation_energy = row['Corrected Energy']
    entry = ComputedEntry(composition, formation_energy)
    mg_entries.append(entry)

  return mg_entries


gnome_grand_diagram_entries = collect_phase_diagram_entries(oxygen_chemsys, grouped_entries, all_crystals)


gnome_grand_diagram = phase_diagram.GrandPotentialPhaseDiagram(
    gnome_grand_diagram_entries,
    chempots=chempots)


sample_entry = ComputedEntry(
    sample['Composition'].item(),
    sample['Corrected Energy'].item(),
)

sample_grand_entry = phase_diagram.GrandPotPDEntry(
    sample_entry,
    chempots=chempots
)


decomposition, decomposition_energy = gnome_grand_diagram.get_decomp_and_e_above_hull(sample_grand_entry)


# The decomposition energy of the provided structure with respect to oxgyen
print(f'Decomposition energy with oxgyen: {decomposition_energy}')
gnome_phase_diagram_entries = collect_phase_diagram_entries(
    chemsys + ('H', 'C', 'O'),
    grouped_entries,
    all_crystals
)
gnome_phase_diagram = phase_diagram.PhaseDiagram(gnome_phase_diagram_entries)


carbon_dioxide = mg.core.Composition('CO2')
gnome_co2_rxns = interface_reactions.InterfacialReactivity(
    sample_entry.composition,
    carbon_dioxide,
    gnome_phase_diagram,
    use_hull_energy=True,
)
co2_stability = gnome_co2_rxns.minimum[1]
co2_stability
water = mg.core.Composition('H2O')
gnome_h2o_rxns = interface_reactions.InterfacialReactivity(
    sample_entry.composition,
    water,
    gnome_phase_diagram,
    use_hull_energy=True,
)
h2o_stability = gnome_h2o_rxns.minimum[1]
h2o_stability

Cloning into 'materials_discovery'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 103 (delta 55), reused 71 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 33.14 MiB | 6.19 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (21/21), done.
Decomposition energy with oxgyen: 1.0495793942046374


-0.056754902370384774